In [1]:
import numpy as np

In [ ]:
# load data

# LF input and output
x_l1 = np.loadtxt('./narrow/matter_power_297_Box100_Part75_27_Box100_Part300_z0/train_input_fidelity_0.txt')
y_l1 = np.loadtxt('./narrow/matter_power_297_Box100_Part75_27_Box100_Part300_z0/train_output_fidelity_0.txt')
k_l1 = np.loadtxt('./narrow/matter_power_297_Box100_Part75_27_Box100_Part300_z0/kf.txt')

x_l2 = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/train_input_fidelity_0.txt')
y_l2 = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/train_output_fidelity_0.txt')
k_l2 = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/kf.txt')

# HF input and output
x_h = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/train_input_fidelity_1.txt')
y_h = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/train_output_fidelity_1.txt')
k_h = np.loadtxt('./narrow/matter_power_297_Box25_Part75_27_Box100_Part300_z0/kf.txt')

In [3]:
# loop over the L1 input and find the ones (indices) that are in the HF input
indices = []
for i in range(len(x_l1)):
    if x_l1[i] in x_h:
        indices.append(i)


In [4]:
indices

[6,
 7,
 8,
 54,
 55,
 56,
 60,
 61,
 62,
 72,
 73,
 74,
 81,
 82,
 83,
 168,
 169,
 170,
 189,
 190,
 191,
 192,
 193,
 194,
 282,
 283,
 284]

In [5]:
k_h

array([-0.59976085, -0.57448928, -0.5492177 , -0.52394612, -0.49867454,
       -0.47340297, -0.44813139, -0.42285981, -0.39758823, -0.37231666,
       -0.34704508, -0.3217735 , -0.29650192, -0.27123035, -0.24595877,
       -0.22068719, -0.19541562, -0.17014404, -0.14487246, -0.11960088,
       -0.09432931, -0.06905773, -0.04378615, -0.01851457,  0.006757  ,
        0.03202858,  0.05730016,  0.08257174,  0.10784331,  0.13311489,
        0.15838647,  0.18365804,  0.20892962,  0.2342012 ,  0.25947278,
        0.28474435,  0.31001593,  0.33528751,  0.36055909,  0.38583066,
        0.41110224,  0.43637382,  0.4616454 ,  0.48691697,  0.51218855,
        0.53746013,  0.5627317 ,  0.58800328,  0.61327486,  0.63854644,
        0.66381801,  0.68908959,  0.71436117,  0.73963275,  0.76490432,
        0.7901759 ,  0.81544748,  0.84071906,  0.86599063,  0.89126221,
        0.91653379,  0.94180536,  0.96707694,  0.99234852])

In [6]:
def cut_and_stitch(k_l1, y_l1, k_l2, y_l2, k_h): # y_l1 and y_l2 are 2D arrays
    # index of middle k value
    middle = len(k_h) // 2
    # interpolate the L1 output to the first half HF k values
    y_l1_interp = np.array([np.interp(k_h[:middle], k_l1, y_l1[i,:]) for i in range(len(y_l1))])
    # interpolate the L1 output to the second half HF k values
    y_l2_interp = np.array([np.interp(k_h[middle:], k_l2, y_l2[i,:]) for i in range(len(y_l2))])
    # stitch the two interpolated outputs together
    y_interp = np.concatenate((y_l1_interp, y_l2_interp), axis=1)
    return y_interp

In [7]:
# cut y_l1 and y_l2 and stitch them together
y_lf = cut_and_stitch(k_l1, y_l1[indices], k_l2, y_l2[indices], k_h)

In [8]:
# create dataset for LF-HF correlation
# input: x_h, y_l1[indices], y_l2[indices] (concatenate)
x_xylf = np.concatenate((x_h, y_lf), axis=1)

# output: y_h 
y_h = y_h

In [9]:
x_xylf.shape

(27, 74)

In [11]:
# save the dataset
save_dir = 'pre_N_L-H_z0'

np.savetxt(save_dir + '/train_input.txt', x_xylf)
np.savetxt(save_dir + '/train_output.txt', y_h)

In [12]:
# save the dataset
save_dir = 'pre_N_L-H_stitch_z0'

np.savetxt(save_dir + '/train_input.txt', y_lf)
np.savetxt(save_dir + '/train_output.txt', y_h)